In [556]:
# import dependencies
import pandas as pd
import time
import json
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '16'

In [560]:
# ingest all data and concatenate partitions
pscs = []
for line in open('/Users/rachelchan/Downloads/psc/CompanyDirectorGraph_4133042451330379265/partition-0.json','r'):
    pscs.append(json.loads(line))
for line in open('/Users/rachelchan/Downloads/psc/CompanyDirectorGraph_4133042451330379265/partition-1.json', 'r'):
    pscs.append(json.loads(line))
for line in open('/Users/rachelchan/Downloads/psc/CompanyDirectorGraph_4133042451330379265/partition-2.json', 'r'):
    pscs.append(json.loads(line))
for line in open('/Users/rachelchan/Downloads/psc/CompanyDirectorGraph_4133042451330379265/partition-3.json', 'r'):
    pscs.append(json.loads(line))
    
pscs

[{'timestamp': 1659571200000, 'window': None, 'row': []},
 {'timestamp': 1659571200000,
  'window': None,
  'row': ['David-Extance-2-1970', None, None, None]},
 {'timestamp': 1659571200000,
  'window': None,
  'row': ['William-Gresswell-12-1991', None, None, None]},
 {'timestamp': 1659571200000, 'window': None, 'row': []},
 {'timestamp': 1659571200000,
  'window': None,
  'row': ['Mary-Anne-Vincent-7-1966', None, None, None]},
 {'timestamp': 1659571200000, 'window': None, 'row': []},
 {'timestamp': 1659571200000,
  'window': None,
  'row': ['Phoenix-James-Limited-00', None, None, None]},
 {'timestamp': 1659571200000,
  'window': None,
  'row': ['Neil-Newman-2-1968', None, None, None]},
 {'timestamp': 1659571200000,
  'window': None,
  'row': ['Weiyao-Sun-8-1983', None, None, None]},
 {'timestamp': 1659571200000, 'window': None, 'row': []},
 {'timestamp': 1659571200000,
  'window': None,
  'row': ['Derek-Patrick-Bernard-Cameron-11-1938',
   ['1459965600000-SC182814-75'],
   {'Scotmas-Gr

In [563]:
pscRawDataframe = pd.DataFrame.from_dict(pscs)
pscRawDataframe.drop('window', inplace=True, axis=1)
pscRawDataframe

,timestamp,row
0,1659571200000,[]
1,1659571200000,"[David-Extance-2-1970, None, None, None]"
2,1659571200000,"[William-Gresswell-12-1991, None, None, None]"
3,1659571200000,[]
4,1659571200000,"[Mary-Anne-Vincent-7-1966, None, None, None]"
...,...,...
1516,1659571200000,"[Hunter/Gatherer-Group-Limited-00, None, None,..."
1517,1659571200000,"[John-Jack-Hodson-2-1980, None, None, None]"
1518,1659571200000,"[Gentex-Europe-Limited-00, [1623434400000-1000..."
1519,1659571200000,[]


In [564]:
splitUpDataframe = pd.DataFrame(pscRawDataframe['row'].tolist(), columns=['psc','time-company-share', 'other_pscs','company_number'])
cleanDataframe=splitUpDataframe.dropna()
cleanDataframe

,psc,time-company-share,other_pscs,company_number
10,Derek-Patrick-Bernard-Cameron-11-1938,[1459965600000-SC182814-75],"{'Scotmas-Group-Ltd-00': 1, 'Alistair-Peter-Ca...",[SC182814]
18,Stephen-Trowbridge-5-1973,[1571162400000-10799086-0],"{'Ian-Roland-Metcalfe-2-1958': 1, 'Theta-Bidco...",[10799086]
33,Practice-Plus-Group-Holdings-Limited-00,[1495735200000-10498997-75],{'Care-Uk-Limited-00': 1},[10498997]
38,Craig-Kenneth-Mckenzie-1-1960,[1467396000000-SC177228-25],{'Kenneth-Ross-Forrest-8-1961': 1},[SC177228]
39,This-Is-Wonderland-Ltd-00,[1459965600000-NI054308-75],{'Timothy-Philip-Rodgers-6-1981': 1},[NI054308]
...,...,...,...,...
1496,James-Philip-Tucker-1-1966,[1615399200000-10610555-0],"{'Philip-David-Jones-7-1961': 1, 'Andrew-James...",[10610555]
1508,Steven-Holmes-5-1985,[1470679200000-10318610-25],{'Hollie-Holmes-11-1984': 1},[10318610]
1513,Michael-John-Godfrey-1-1957,[1573581600000-SC646966-75],{'Amt2-Limited-00': 1},[SC646966]
1514,Robert-Copeland-9-1981,[1463767200000-SC399884-25],"{'Patrick-Michael-Byrne-5-1980': 1, '4-Icg-Ltd...",[SC399884]


In [565]:
# extract other_pscs column out of dictionary format
new_rows=[]

for i,row in cleanDataframe.iterrows():
    for key, value in row['other_pscs'].items():
        temp_row={
            'psc': row['psc'],
            'time-company-share': row['time-company-share'],
            'company_number': row['company_number'],
            'other_psc': key,
            'other_psc_count': value
        }
        new_rows.append(temp_row)  
pscdf = pd.DataFrame(new_rows)

In [567]:
# Get company number out of brackets
pscdf = pscdf.explode('company_number')

In [570]:
# extract time-company-share column into its own separate columns
new_rows=[]

for i,row in pscdf.iterrows():
    company_number = row['company_number']
    foundtime = None
    for triplet in row['time-company-share']:
        timeA, companyA, shareA = triplet.split("-")
        if (companyA == company_number):
            foundtime = timeA
        temp_row={
            'psc': row['psc'],
            'time': foundtime,
            'company_number': row['company_number'],
            'other_psc': row['other_psc'],
            'share_ownership': shareA,
            'other_psc_count': row['other_psc_count']
        }
        new_rows.append(temp_row)  
finaldf = pd.DataFrame(new_rows).drop_duplicates()
finaldf

,psc,time,company_number,other_psc,share_ownership,other_psc_count
0,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Scotmas-Group-Ltd-00,75,1
1,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Alistair-Peter-Cameron-11-1977,75,1
2,Stephen-Trowbridge-5-1973,1571162400000,10799086,Ian-Roland-Metcalfe-2-1958,0,1
3,Stephen-Trowbridge-5-1973,1571162400000,10799086,Theta-Bidco-Limited-00,0,1
4,Stephen-Trowbridge-5-1973,1571162400000,10799086,Piers-Leigh-Stuart-Wilson-9-1967,0,1
...,...,...,...,...,...,...
511,Robert-Copeland-9-1981,1463767200000,SC399884,Patrick-Michael-Byrne-5-1980,25,1
512,Robert-Copeland-9-1981,1463767200000,SC399884,4-Icg-Ltd-00,25,1
513,Gentex-Europe-Limited-00,1623434400000,10002483,Design-Reality-Ip-Limited-00,75,1
514,Gentex-Europe-Limited-00,1623434400000,10002483,Troy-Alan-Baker-8-1977,75,1


In [571]:
# drop None values
finaldf = finaldf.dropna()
finaldf

,psc,time,company_number,other_psc,share_ownership,other_psc_count
0,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Scotmas-Group-Ltd-00,75,1
1,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Alistair-Peter-Cameron-11-1977,75,1
2,Stephen-Trowbridge-5-1973,1571162400000,10799086,Ian-Roland-Metcalfe-2-1958,0,1
3,Stephen-Trowbridge-5-1973,1571162400000,10799086,Theta-Bidco-Limited-00,0,1
4,Stephen-Trowbridge-5-1973,1571162400000,10799086,Piers-Leigh-Stuart-Wilson-9-1967,0,1
...,...,...,...,...,...,...
511,Robert-Copeland-9-1981,1463767200000,SC399884,Patrick-Michael-Byrne-5-1980,25,1
512,Robert-Copeland-9-1981,1463767200000,SC399884,4-Icg-Ltd-00,25,1
513,Gentex-Europe-Limited-00,1623434400000,10002483,Design-Reality-Ip-Limited-00,75,1
514,Gentex-Europe-Limited-00,1623434400000,10002483,Troy-Alan-Baker-8-1977,75,1


In [575]:
# Remove duplicates regardless of order
sortedFinalDf = finaldf[~pd.DataFrame(np.sort(finaldf[['psc','time','company_number','other_psc']], axis=1), index=finaldf.index).duplicated()]
sortedFinalDf

,psc,time,company_number,other_psc,share_ownership,other_psc_count
0,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Scotmas-Group-Ltd-00,75,1
1,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Alistair-Peter-Cameron-11-1977,75,1
2,Stephen-Trowbridge-5-1973,1571162400000,10799086,Ian-Roland-Metcalfe-2-1958,0,1
3,Stephen-Trowbridge-5-1973,1571162400000,10799086,Theta-Bidco-Limited-00,0,1
4,Stephen-Trowbridge-5-1973,1571162400000,10799086,Piers-Leigh-Stuart-Wilson-9-1967,0,1
...,...,...,...,...,...,...
508,James-Philip-Tucker-1-1966,1615399200000,10610555,Andrew-James-Brear-1-1960,0,1
512,Robert-Copeland-9-1981,1463767200000,SC399884,4-Icg-Ltd-00,25,1
513,Gentex-Europe-Limited-00,1623434400000,10002483,Design-Reality-Ip-Limited-00,75,1
514,Gentex-Europe-Limited-00,1623434400000,10002483,Troy-Alan-Baker-8-1977,75,1


In [576]:
# Get PSCs with greater than 5 companies
greaterThanFiveDf = df[df.groupby("company_number")['psc'].transform('size') > 5]
greaterThanFiveDf

,psc,time,company_number,other_psc,share_ownership,other_psc_count
2,Stephen-Trowbridge-5-1973,1571162400000,10799086,Ian-Roland-Metcalfe-2-1958,0,1
3,Stephen-Trowbridge-5-1973,1571162400000,10799086,Theta-Bidco-Limited-00,0,1
4,Stephen-Trowbridge-5-1973,1571162400000,10799086,Piers-Leigh-Stuart-Wilson-9-1967,0,1
5,Stephen-Trowbridge-5-1973,1571162400000,10799086,Kenneth-George-Hanna-4-1953,0,1
6,Stephen-Trowbridge-5-1973,1571162400000,10799086,Gregory-Michael-Lawless-6-1960,0,1
...,...,...,...,...,...,...
507,James-Philip-Tucker-1-1966,1615399200000,10610555,Philip-David-Jones-7-1961,0,1
508,James-Philip-Tucker-1-1966,1615399200000,10610555,Andrew-James-Brear-1-1960,0,1
513,Gentex-Europe-Limited-00,1623434400000,10002483,Design-Reality-Ip-Limited-00,75,1
514,Gentex-Europe-Limited-00,1623434400000,10002483,Troy-Alan-Baker-8-1977,75,1


In [592]:
# Get PSCs with greater than 1 company
greaterThanOneDf = df[df.groupby("company_number")['psc'].transform('size') > 1]
greaterThanOneDf

,psc,time,company_number,other_psc,share_ownership,other_psc_count
0,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Scotmas-Group-Ltd-00,75,1
1,Derek-Patrick-Bernard-Cameron-11-1938,1459965600000,SC182814,Alistair-Peter-Cameron-11-1977,75,1
2,Stephen-Trowbridge-5-1973,1571162400000,10799086,Ian-Roland-Metcalfe-2-1958,0,1
3,Stephen-Trowbridge-5-1973,1571162400000,10799086,Theta-Bidco-Limited-00,0,1
4,Stephen-Trowbridge-5-1973,1571162400000,10799086,Piers-Leigh-Stuart-Wilson-9-1967,0,1
...,...,...,...,...,...,...
508,James-Philip-Tucker-1-1966,1615399200000,10610555,Andrew-James-Brear-1-1960,0,1
512,Robert-Copeland-9-1981,1463767200000,SC399884,4-Icg-Ltd-00,25,1
513,Gentex-Europe-Limited-00,1623434400000,10002483,Design-Reality-Ip-Limited-00,75,1
514,Gentex-Europe-Limited-00,1623434400000,10002483,Troy-Alan-Baker-8-1977,75,1


In [593]:
# Convert time column from epoch to datetime
greaterThanOneDf['time']=pd.to_datetime(greaterThanOneDf['time'], unit='ms')\
                 .dt.tz_localize('UTC' )\
                 .dt.tz_convert('Europe/London')

/var/folders/0l/f4m1gtfj2_30yxgdpqqlvxkc0000gn/T/ipykernel_29862/227572496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  greaterThanOneDf['time']=pd.to_datetime(greaterThanOneDf['time'], unit='ms')\


In [594]:
# Remove time from date
greaterThanOneDf['date'] = pd.to_datetime(greaterThanOneDf['time']).dt.date

/var/folders/0l/f4m1gtfj2_30yxgdpqqlvxkc0000gn/T/ipykernel_29862/2121770678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  greaterThanOneDf['date'] = pd.to_datetime(greaterThanOneDf['time']).dt.date


In [595]:
# Drop other_psc_count and time column as this is not needed for our visualisation
greaterThanOneDf = greaterThanOneDf.drop('other_psc_count', axis=1)
greaterThanOneDf = greaterThanOneDf.drop('time', axis=1)
greaterThanOneDf

,psc,company_number,other_psc,share_ownership,date
0,Derek-Patrick-Bernard-Cameron-11-1938,SC182814,Scotmas-Group-Ltd-00,75,2016-04-06
1,Derek-Patrick-Bernard-Cameron-11-1938,SC182814,Alistair-Peter-Cameron-11-1977,75,2016-04-06
2,Stephen-Trowbridge-5-1973,10799086,Ian-Roland-Metcalfe-2-1958,0,2019-10-15
3,Stephen-Trowbridge-5-1973,10799086,Theta-Bidco-Limited-00,0,2019-10-15
4,Stephen-Trowbridge-5-1973,10799086,Piers-Leigh-Stuart-Wilson-9-1967,0,2019-10-15
...,...,...,...,...,...
508,James-Philip-Tucker-1-1966,10610555,Andrew-James-Brear-1-1960,0,2021-03-10
512,Robert-Copeland-9-1981,SC399884,4-Icg-Ltd-00,25,2016-05-20
513,Gentex-Europe-Limited-00,10002483,Design-Reality-Ip-Limited-00,75,2021-06-11
514,Gentex-Europe-Limited-00,10002483,Troy-Alan-Baker-8-1977,75,2021-06-11


In [419]:
# Convert date column to string type
greaterThanOneDf['date'] = greaterThanOneDf['date'].astype(str)

In [621]:
greaterThanOneDf['company_number'] = greaterThanOneDf['company_number'].replace(['SC399884'], 'Pursuit Marketing Limited')

In [624]:
# Using pyvis to visualise a psc to company network graph with edge weights labelled with date and share ownership, 
# thickness of edge corresponds to share ownership percentage.

from pyvis.network import Network
nt = Network('800px', '800px',bgcolor='#white', font_color='black')
pscs = greaterThanOneDf['psc']
otherpscs = greaterThanOneDf['other_psc']
companies = greaterThanOneDf['company_number']
share = greaterThanOneDf['share_ownership']
date = greaterThanOneDf['date']

edge_data = zip(pscs, otherpscs, share, companies, date)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
    newdst = e[3]
    date = e[4]
    
    nt.add_node(src, src, title=src,shape='image', image ="https://www.seekpng.com/png/small/846-8465978_critres-de-choix-pour-la-solution-de-cration.png")
    nt.add_node(dst, dst, title=dst, shape='image', image ="https://www.seekpng.com/png/small/846-8465978_critres-de-choix-pour-la-solution-de-cration.png")
    nt.add_node(newdst,newdst,title=newdst, shape='image', image="https://cdn-icons-png.flaticon.com/512/2083/2083417.png")
    nt.add_edge(src, newdst, value=w, title=f'Ownership: {w}%, Notified On: {date}', color='#F6E1D3')
    nt.add_edge(dst, newdst, value=w, title=f'Ownership: {w}%, Notified On: {date}', color='#F6E1D3')
        
nt.show_buttons(filter_=['physics'])
nt.show('nx.html')   

In [625]:
IFrame(src='nx.html', width=1000, height=1000)